In [3]:
import cv2
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint

def create_drowsiness_model(input_shape):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
    return model



In [5]:
def train_model():
    train_data_dir = 'project/d'  
    test_data_dir = 'project/dataset_new/test/Open/'   
    input_shape = (24, 24, 3)
    model = create_drowsiness_model(input_shape)

    train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True
    )

    test_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(24, 24),
        batch_size=32,
        class_mode='binary'
    )

    test_generator = test_datagen.flow_from_directory(
        test_data_dir,
        target_size=(24, 24),
        batch_size=32,
        class_mode='binary'
    )

    print("Number of training samples:", len(train_generator.filenames))
    print("Number of testing samples:", len(test_generator.filenames))

    if len(train_generator) == 0 or len(test_generator) == 0:
        print("No images found in the specified directories.")
        return

    checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)

    model.fit(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=10,
        validation_data=test_generator,
        validation_steps=len(test_generator),
        callbacks=[checkpoint]
    )




In [6]:
if __name__ == "__main__":
    train_model()

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'project/dataset_new/train/Open/'